In [47]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Daily, Hourly
import warnings

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [52]:
races = pd.read_csv(path+'data/races.csv')
results = pd.read_csv(path+'data/results.csv')

weather = races.iloc[:,[0,1,2,3,4,5,7]]
weather.shape

(160, 7)

In [49]:
def format_datetime(dt):
    year, time = dt.split(' ')

    return datetime(*map(int, year.split('-')), *map(int, time.split(':')))

In [53]:
weather.date = weather.date.apply(lambda x: format_datetime(x))

In [55]:
def get_weather_code(dt):
    print(dt)
    return dt

In [56]:
weather['code'] = weather.date.apply(lambda x: get_weather_code(x))

2014-03-16 06:00:00
2014-03-30 08:00:00
2014-04-06 15:00:00
2014-04-20 07:00:00
2014-05-11 12:00:00
2014-05-25 12:00:00
2014-06-08 18:00:00
2014-06-22 12:00:00
2014-07-06 12:00:00
2014-07-20 12:00:00
2014-07-27 12:00:00
2014-08-24 12:00:00
2014-09-07 12:00:00
2014-09-21 12:00:00
2014-10-05 06:00:00
2014-10-12 11:00:00
2014-11-02 20:00:00
2014-11-09 16:00:00
2014-11-23 13:00:00
2015-03-15 05:00:00
2015-03-29 07:00:00
2015-04-12 06:00:00
2015-04-19 15:00:00
2015-05-10 12:00:00
2015-05-24 12:00:00
2015-06-07 18:00:00
2015-06-21 12:00:00
2015-07-05 12:00:00
2015-07-26 12:00:00
2015-08-23 12:00:00
2015-09-06 12:00:00
2015-09-20 12:00:00
2015-09-27 05:00:00
2015-10-11 11:00:00
2015-10-25 19:00:00
2015-11-01 19:00:00
2015-11-15 16:00:00
2015-11-29 13:00:00
2016-03-20 05:00:00
2016-04-03 15:00:00
2016-04-17 06:00:00
2016-05-01 12:00:00
2016-05-15 12:00:00
2016-05-29 12:00:00
2016-06-12 18:00:00
2016-06-19 13:00:00
2016-07-03 12:00:00
2016-07-10 12:00:00
2016-07-24 12:00:00
2016-07-31 12:00:00


In [58]:
weather.head()

,season,round,circuit_id,country,lat,long,date,code
0,2014,1,albert_park,Australia,-37.84970,144.96800,2014-03-16 06:00:00,2014-03-16 06:00:00
1,2014,2,sepang,Malaysia,2.76083,101.73800,2014-03-30 08:00:00,2014-03-30 08:00:00
2,2014,3,bahrain,Bahrain,26.03250,50.51060,2014-04-06 15:00:00,2014-04-06 15:00:00
3,2014,4,shanghai,China,31.33890,121.22000,2014-04-20 07:00:00,2014-04-20 07:00:00
4,2014,5,catalunya,Spain,41.57000,2.26111,2014-05-11 12:00:00,2014-05-11 12:00:00


In [ ]:
start = datetime(2018,1,1)
end = datetime(2018,1,28)
coord = Point(race.lat, race.long)

data = Daily(coord, start, end).fetch()

data.plot(y=['tavg'])
print(data.columns)
plt.show()

In [ ]:
def format_coco(coco):
    codes = {
        1.0 :'Clear',
        2.0 : 'Fair',
        3.0 : 'Cloudy',
        4.0 : 'Overcast',
        5.0 : 'Fog',
        6.0 : 'Freezing Fog',
        7.0 : 'Light Rain',
        8.0 : 'Rain',
        9.0 : 'Heavy Rain',
        10.0 : 'Freezing Rain',
        11.0 : 'Heavy Freezing Rain',
        12.0 : 'Sleet',
        13.0 : 'Heavy Sleet',
        14.0 : 'Light Snowfall',
        15.0 : 'Snowfall',
        16.0 : 'Heavy Snowfall',
        17.0 : 'Rain Shower',
        18.0 : 'Heavy Rain Shower',
        19.0 : 'Sleet Shower',
        20.0 : 'Heavy Sleet Shower',
        21.0 : 'Snow Shower',
        22.0 : 'Heavy Snow Shower',
        23.0 : 'Lightning',
        24.0 : 'Hail',
        25.0 : 'Thunderstorm',
        26.0 : 'Heavy Thunderstorm',
        27.0 : 'Storm',
    }
    return codes[coco]

In [ ]:
# start = datetime.strptime(race.date, '%Y-%m-%d')
# end = datetime.strptime(race.date, '%Y-%m-%d')
start = datetime(2021,1,1)
end = datetime(2021,12,31,23)
coord = Point(race.lat, race.long)

data = Hourly(coord, start, end).fetch()

weather = data.groupby('coco').rhum.describe().reset_index()
weather['coco'] = weather.coco.apply(lambda x: format_coco(x))

In [ ]:
results.head()

In [ ]:
weather.head()

In [ ]:
data.plot(y=['temp'])
plt.show()